In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np

class ImageProcessorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Xử lý ảnh đơn giản")
        self.root.geometry("1200x600")

        # Ảnh gốc và ảnh xử lý
        self.original_image = None
        self.processed_image = None
        self.display_image_original = None
        self.display_image_processed = None

        # Tạo khung chứa hai Canvas
        self.image_frame = tk.Frame(root)
        self.image_frame.pack(pady=10)

        # Khung hiển thị ảnh gốc
        # Thêm nhãn "Ảnh gốc" phía trên canvas
        self.label_original = tk.Label(self.image_frame, text="Ảnh gốc", font=("Arial", 14))
        self.label_original.pack(side=tk.LEFT, padx=10)

        self.canvas_original = tk.Canvas(self.image_frame, width=500, height=400, bg='gray')
        self.canvas_original.pack(side=tk.LEFT, padx=10)

        # Khung hiển thị ảnh xử lý (ẩn lúc đầu)
        self.canvas_processed = None
        self.label_processed = None  # Nhãn cho ảnh xử lý, sẽ được tạo sau

        # Các nút chức năng
        btn_frame = tk.Frame(root)
        btn_frame.pack(pady=10)

        tk.Button(btn_frame, text="Nhập ảnh", command=self.load_image).grid(row=0, column=0, padx=5)
        tk.Button(btn_frame, text="Làm rõ ảnh", command=self.sharpen_image).grid(row=0, column=1, padx=5)
        tk.Button(btn_frame, text="Khử mờ", command=self.deblur_image).grid(row=0, column=2, padx=5)
        tk.Button(btn_frame, text="Quay lại ảnh gốc", command=self.restore_image).grid(row=0, column=3, padx=5)
        tk.Button(btn_frame, text="Xóa ảnh", command=self.clear_image).grid(row=0, column=4, padx=5)

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg *.bmp")])
        if file_path:
            image = cv2.imread(file_path)
            if image is None:
                messagebox.showerror("Lỗi", "Không thể đọc được tệp ảnh. Vui lòng chọn tệp hợp lệ.")
                return
            self.original_image = image.copy()
            self.processed_image = image.copy()
            self.display_image_on_canvas()

    def display_image_on_canvas(self):
        # Hiển thị ảnh gốc
        if self.original_image is not None:
            if len(self.original_image.shape) == 3 and self.original_image.shape[2] == 3:
                image = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB)
            else:
                image = self.original_image
            image = Image.fromarray(image)
            image.thumbnail((500, 400))
            self.display_image_original = ImageTk.PhotoImage(image)
            self.canvas_original.create_image(250, 200, image=self.display_image_original)

        # Hiển thị ảnh xử lý nếu khung xử lý tồn tại
        if self.canvas_processed and self.processed_image is not None:
            if len(self.processed_image.shape) == 3 and self.processed_image.shape[2] == 3:
                image = cv2.cvtColor(self.processed_image, cv2.COLOR_BGR2RGB)
            else:
                image = self.processed_image
            image = Image.fromarray(image)
            image.thumbnail((500, 400))
            self.display_image_processed = ImageTk.PhotoImage(image)
            self.canvas_processed.create_image(250, 200, image=self.display_image_processed)

    def create_processed_canvas(self):
        """Tạo khung hiển thị ảnh xử lý nếu chưa có."""
        if not self.canvas_processed:
            # Thêm nhãn "Ảnh đã xử lý" phía trên canvas
            self.label_processed = tk.Label(self.image_frame, text="Ảnh đã xử lý", font=("Arial", 14))
            self.label_processed.pack(side=tk.LEFT, padx=10)
            
            self.canvas_processed = tk.Canvas(self.image_frame, width=500, height=400, bg='gray')
            self.canvas_processed.pack(side=tk.LEFT, padx=10)

    def sharpen_image(self):
        if self.processed_image is not None:
            if len(self.processed_image.shape) != 3 or self.processed_image.shape[2] != 3:
                messagebox.showerror("Lỗi", "Ảnh phải là ảnh màu (RGB).")
                return
            kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
            sharpened = cv2.filter2D(self.processed_image, -1, kernel)
            self.processed_image = sharpened
            self.create_processed_canvas()
            self.display_image_on_canvas()
        else:
            messagebox.showwarning("Cảnh báo", "Chưa có ảnh để xử lý.")

    def deblur_image(self):
        if self.processed_image is not None:
            if len(self.processed_image.shape) != 3 or self.processed_image.shape[2] != 3:
                messagebox.showerror("Lỗi", "Ảnh phải là ảnh màu (RGB).")
                return
            kernel = np.array([[-1, -1, -1],
                               [-1,  9, -1],
                               [-1, -1, -1]])
            deblurred = cv2.filter2D(self.processed_image, -1, kernel)
            self.processed_image = deblurred
            self.create_processed_canvas()
            self.display_image_on_canvas()
        else:
            messagebox.showwarning("Cảnh báo", "Chưa có ảnh để xử lý.")

    def restore_image(self):
        if self.original_image is not None:
            self.processed_image = self.original_image.copy()
            if self.canvas_processed:
                self.canvas_processed.destroy()
                self.canvas_processed = None
                if self.label_processed:
                    self.label_processed.destroy()
                    self.label_processed = None
            self.display_image_on_canvas()
        else:
            messagebox.showwarning("Cảnh báo", "Không có ảnh gốc để khôi phục.")

    def clear_image(self):
        if self.original_image is not None or self.processed_image is not None:
            self.original_image = None
            self.processed_image = None
            self.canvas_original.delete("all")
            if self.canvas_processed:
                self.canvas_processed.destroy()
                self.canvas_processed = None
                if self.label_processed:
                    self.label_processed.destroy()
                    self.label_processed = None
        else:
            messagebox.showwarning("Cảnh báo", "Không có ảnh để xóa.")

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageProcessorApp(root)
    root.mainloop()

Note: you may need to restart the kernel to use updated packages.
